# Validating Historical Features with Great Expectations

![Validating Historical Features with Great Expectations](./dqm-diagram-1.png)

In this tutorial, we will use the public dataset of Chicago taxi trips to present data validation capabilities of Feast. The original dataset is stored in BigQuery and consists of raw data for each taxi trip (one row per trip) since 2013. We will generate several training datasets (aka historical features in Feast) for different periods and evaluate expectations made on one dataset against another. Our features will represent aggregations of raw data with daily intervals (eg, trips per day, average fare or speed for a specific day, etc.). We will craft some features using SQL while pulling data from BigQuery (like total trips time or total miles travelled). Another chunk of features will be implemented using Feast's on-demand transformations - features calculated on the fly when requested.

Our plan:

0. Prepare environment
1. Pull data from BigQuery (optional)
2. Declare & apply features and feature views in Feast
3. Generate reference dataset
4. Develop & test profiler function
5. Run validation on different dataset using reference dataset & profiler

### 0. Setup

Install Feast Python SDK and great expectations:

In [1]:
#!pip install 'feast[ge]'

### 1. Dataset preparation (Optional) 

**You can skip this step if you don't have GCP account. Please use parquet files that are coming with this tutorial instead**

!pip install google-cloud-bigquery
import pyarrow.parquet
from google.cloud.bigquery import Client
bq_client = Client(project='kf-feast')
Running some basic aggregations while pulling data from BigQuery. Grouping by taxi_id and day:
data_query = """SELECT
    taxi_id,
    TIMESTAMP_TRUNC(trip_start_timestamp, DAY) as day,
    SUM(trip_miles) as total_miles_travelled,
    SUM(trip_seconds) as total_trip_seconds,
    SUM(fare) as total_earned,
    COUNT(*) as trip_count
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    trip_miles > 0 AND trip_seconds > 60 AND
    trip_start_timestamp BETWEEN '2019-01-01' and '2020-12-31' AND
    trip_total < 1000
GROUP BY taxi_id, TIMESTAMP_TRUNC(trip_start_timestamp, DAY)"""
driver_stats_table = bq_client.query(data_query).to_arrow()

# Storing resulting dataset into parquet file
pyarrow.parquet.write_table(driver_stats_table, "trips_stats.parquet")
def entities_query(year):
    return f"""SELECT
    distinct taxi_id
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    trip_miles > 0 AND trip_seconds > 0 AND
    trip_start_timestamp BETWEEN '{year}-01-01' and '{year}-12-31'
"""
entities_2019_table = bq_client.query(entities_query(2019)).to_arrow()

# Storing entities (taxi ids) into parquet file
pyarrow.parquet.write_table(entities_2019_table, "entities.parquet")
#entities_2020_table = bq_client.query(entities_query(2020)).to_arrow()
#pyarrow.parquet.write_table(entities_2019_table, "entities_2020.parquet")

## 2. Declaring features

In [2]:
import pyarrow.parquet
import pandas as pd
from feast import Feature, FeatureView, Entity, FeatureStore
from feast.value_type import ValueType
from feast.data_format import ParquetFormat
from feast.on_demand_feature_view import on_demand_feature_view
from feast.infra.offline_stores.file_source import FileSource
from feast.infra.offline_stores.file import SavedDatasetFileStorage
from google.protobuf.duration_pb2 import Duration

In [3]:
df = pd.read_parquet("trips_stats.parquet")
df.head(5)

,taxi_id,day,total_miles_travelled,total_trip_seconds,total_earned,trip_count
0,9b2a23587b25fe43998076b3761b979ee91f9f5ac8bdac...,2019-04-30 00:00:00+00:00,31.04,15523,207.00,31
1,26211b46f6f6c6987bee5e2062934ba8692bced4811d1a...,2019-04-05 00:00:00+00:00,88.21,16757,331.50,33
2,018adeb3e0e0f1b8c78f37c4b99735005ea7acfe800ab8...,2020-02-05 00:00:00+00:00,39.69,14331,233.00,36
3,c8549ac4a2842a038f2a5b91ecf894ca7cb04d60558be3...,2020-02-26 00:00:00+00:00,46.74,15535,235.25,30
4,2780ead18beaa862cc67315ddabd9d1acaadcd6da82eba...,2020-02-20 00:00:00+00:00,86.40,21749,353.25,38


In [26]:
df[df.taxi_id == 'd13c5aaa066f94b4927779ed24cd313b0c686f03407095cf5daa4c227d411eb5049a5b1ce8ff07157f9641e0e863be971c9d24bc3ef3f89857d7788fef397242'].sort_values(by="day")

,taxi_id,day,total_miles_travelled,total_trip_seconds,total_earned,trip_count
746606,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-01-02 00:00:00+00:00,17.8,3420,67.50,7
1484460,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-01-03 00:00:00+00:00,25.3,7860,132.50,19
877317,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-01-04 00:00:00+00:00,32.4,6540,116.00,9
109519,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-01-05 00:00:00+00:00,1.2,540,10.25,2
984363,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-01-07 00:00:00+00:00,11.4,3780,66.75,10
...,...,...,...,...,...,...
922475,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2020-03-09 00:00:00+00:00,21.8,6300,89.75,9
1118291,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2020-03-10 00:00:00+00:00,21.8,6360,101.00,12
1008498,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2020-03-11 00:00:00+00:00,14.9,4800,76.50,10
1068987,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2020-03-12 00:00:00+00:00,23.2,6420,99.50,11


In [4]:
batch_source = FileSource(
    event_timestamp_column="day",
    path="trips_stats.parquet",  # using parquet file that we created on previous step
    file_format=ParquetFormat()
)
taxi_entity = Entity(name='taxi', join_key='taxi_id')
trips_stats_fv = FeatureView(
    name='trip_stats',
    entities=['taxi'],
    features=[
        Feature("total_miles_travelled", ValueType.DOUBLE),
        Feature("total_trip_seconds", ValueType.DOUBLE),
        Feature("total_earned", ValueType.DOUBLE),
        Feature("trip_count", ValueType.INT64),

    ],
    ttl=Duration(seconds=86400), # 86400 sec = 1day
    batch_source=batch_source,
)

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/data_source.py:229: DeprecationWarning: The argument 'event_timestamp_column' is being deprecated. Please use 'timestamp_field' instead. instead. Feast 0.23 and onwards will not support the argument 'event_timestamp_column' for datasources.
  warnings.warn(
/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/entity.py:110: DeprecationWarning: The `join_key` parameter is being deprecated in favor of the `join_keys` parameter. Please switch from using `join_key` to `join_keys`. Feast 0.23 and onwards will not support the `join_key` parameter.
  warnings.warn(
/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/feature_view.py:256: DeprecationWarning: batch_source and stream_source have been deprecated in favor of `source`.The deprecated fields will be removed in Feast 0.23.
  warnings.warn(
/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/feature_view.py:

*Read more about feature views in [Feast docs](https://docs.feast.dev/getting-started/concepts/feature-view)*

In [7]:
@on_demand_feature_view(
    features=[
        Feature("avg_fare", ValueType.DOUBLE),
        Feature("avg_speed", ValueType.DOUBLE),
        Feature("avg_trip_seconds", ValueType.DOUBLE),
        Feature("earned_per_hour", ValueType.DOUBLE),
    ],
    sources=[trips_stats_fv]
)

def on_demand_stats(inp):
    out = pd.DataFrame()
    out["avg_fare"] = inp["total_earned"] / inp["trip_count"]
    out["avg_speed"] = 3600 * inp["total_miles_travelled"] / inp["total_trip_seconds"]
    out["avg_trip_seconds"] = inp["total_trip_seconds"] / inp["trip_count"]
    out["earned_per_hour"] = 3600 * inp["total_earned"] / inp["total_trip_seconds"]
    return out

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/on_demand_feature_view.py:559: DeprecationWarning: The `features` parameter is being deprecated in favor of the `schema` parameter. Please switch from using `features` to `schema`. This will also requiring switching feature definitions from using `Feature` to `Field`. Feast 0.23 and onwards will not support the `features` parameter.
  warnings.warn(


*Read more about on demand feature views [here](https://docs.feast.dev/reference/alpha-on-demand-feature-view)*

In [8]:
store = FeatureStore(".")  # using feature_store.yaml that stored in the same directory

In [9]:
store.apply([taxi_entity, trips_stats_fv, on_demand_stats])  # writing to the registry

## 3. Generating training (reference) dataset

In [10]:
taxi_ids = pyarrow.parquet.read_table("entities.parquet").to_pandas()

Generating range of timestamps with daily frequency:

In [11]:
taxi_ids.head(5)

,taxi_id
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...
1,9d5308a039a77963ed44f4b4435abd45f28eb7c4eb7669...
2,2d411799ad37a21a76e73cab9a4ec254f551ba45761f4e...
3,a36483657911ff455df8757d429c4c7cc1d720b6ef8f4a...
4,e97ff21e0c8f655aa03ec62fbccdf33578a21516535ce7...


In [15]:
taxi_ids.shape

(5064, 1)

In [13]:
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2019-06-01", "2019-07-01", freq='D')
timestamps.head(5)

,event_timestamp
0,2019-06-01
1,2019-06-02
2,2019-06-03
3,2019-06-04
4,2019-06-05


In [16]:
timestamps.shape

(31, 1)

Cross merge (aka relation multiplication) produces entity dataframe with each taxi_id repeated for each timestamp:

In [14]:
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df.head(5)

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-05


In [17]:
entity_df.shape

(156984, 2)

In [18]:
timestamps.shape[0] * taxi_ids.shape[0] # entity_df is the Cartesian product those dataframes

156984

Retrieving historical features for resulting entity dataframe and persisting output as a saved dataset:

In [19]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

store.create_saved_dataset(
    from_=job,
    name='my_training_ds',
    storage=SavedDatasetFileStorage(path='my_training_ds.parquet')
)

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/feature_store.py:952: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


<SavedDataset(name = my_training_ds, features = ['trip_stats:total_miles_travelled', 'trip_stats:total_trip_seconds', 'trip_stats:total_earned', 'trip_stats:trip_count', 'on_demand_stats:avg_fare', 'on_demand_stats:avg_trip_seconds', 'on_demand_stats:avg_speed', 'on_demand_stats:earned_per_hour'], join_keys = ['taxi_id'], storage = <feast.infra.offline_stores.file_source.SavedDatasetFileStorage object at 0x7f9752931340>, full_feature_names = False, tags = {}, feature_service_name = None, _retrieval_job = <feast.infra.offline_stores.file.FileRetrievalJob object at 0x7f9770643d90>, min_event_timestamp = 2019-06-01 00:00:00, max_event_timestamp = 2019-07-01 00:00:00, created_timestamp = 2022-06-08 13:36:29.652232, last_updated_timestamp = 2022-06-08 13:36:29.652232)>

## 4. Developing dataset profiler

Dataset profiler is a function that accepts dataset and generates set of its characteristics. This charasteristics will be then used to evaluate (validate) next datasets.

**Important: datasets are not compared to each other! 
Feast use a reference dataset and a profiler function to generate a reference profile. 
This profile will be then used during validation of the tested dataset.**

In [20]:
import numpy as np

from feast.dqm.profilers.ge_profiler import ge_profiler

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/great_expectations/marshmallow__shade/__init__.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  __version_info__ = tuple(LooseVersion(__version__).version)


Loading saved dataset first and exploring the data:

In [21]:
ds = store.get_saved_dataset('my_training_ds')
ds.to_df()

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/feature_store.py:1005: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,avg_trip_seconds,total_trip_seconds,trip_count,earned_per_hour,event_timestamp,total_earned,avg_fare,avg_speed,total_miles_travelled,taxi_id
41641152,2010.000000,16080,8,45.559701,2019-06-01 00:00:00+00:00,203.50,25.437500,15.559701,69.50,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
17019776,1476.000000,7380,5,36.219512,2019-06-01 00:00:00+00:00,74.25,14.850000,7.707317,15.80,33164e16dd29b1c58cd15cce31df4bfcb75d9903cb66de...
38527755,1270.000000,7620,6,54.212598,2019-06-01 00:00:00+00:00,114.75,19.125000,18.188976,38.50,226fe0b00be42932bdff81bc0b318b883bfbf15dd48093...
17007652,1415.000000,5660,4,45.000000,2019-06-01 00:00:00+00:00,70.75,17.687500,12.860777,20.22,5a5bed1b5ced617d0594007d591f10bbbca354d50b19ca...
14512479,1395.600000,6978,5,53.783319,2019-06-01 00:00:00+00:00,104.25,20.850000,17.793637,34.49,b7f7dbb452c0fb980a0f2050a146147c1006fe5f34e3b0...
...,...,...,...,...,...,...,...,...,...,...
17469489,1205.000000,4820,4,76.369295,2019-07-01 00:00:00+00:00,102.25,25.562500,27.619917,36.98,961263722c1beadafef2355412d672acac35e4054f6aaa...
48525288,692.727273,7620,11,52.677165,2019-07-01 00:00:00+00:00,111.50,10.136364,13.700787,29.00,8b07f9156e568a37d362463c84dbd1118b4eeb753bae50...
5295296,588.750000,9420,16,54.649682,2019-07-01 00:00:00+00:00,143.00,8.937500,11.847134,31.00,a112879f10892d5c698ce150af17aa28615b6d005ca749...
46508066,1647.000000,4941,3,73.770492,2019-07-01 00:00:00+00:00,101.25,33.750000,27.584699,37.86,68fe14b9fc2d53de5ac349d47f80f43fea895e201a31e3...


In [24]:
ds.to_df()[ds.to_df().taxi_id == 'd13c5aaa066f94b4927779ed24cd313b0c686f03407095cf5daa4c227d411eb5049a5b1ce8ff07157f9641e0e863be971c9d24bc3ef3f89857d7788fef397242']

,avg_trip_seconds,total_trip_seconds,trip_count,earned_per_hour,event_timestamp,total_earned,avg_fare,avg_speed,total_miles_travelled,taxi_id
41641152,2010.000000,16080,8,45.559701,2019-06-01 00:00:00+00:00,203.50,25.437500,15.559701,69.5,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41641378,1240.000000,3720,3,57.822581,2019-06-02 00:00:00+00:00,59.75,19.916667,19.451613,20.1,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41641758,2014.285714,14100,7,57.191489,2019-06-03 00:00:00+00:00,224.00,32.000000,21.370213,83.7,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41642063,2014.285714,14100,7,57.191489,2019-06-04 00:00:00+00:00,224.00,32.000000,21.370213,83.7,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41642370,1954.285714,13680,7,51.973684,2019-06-05 00:00:00+00:00,197.50,28.214286,18.947368,72.0,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41642621,2664.000000,13320,5,59.324324,2019-06-06 00:00:00+00:00,219.50,43.900000,23.243243,86.0,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41643076,595.384615,7740,13,55.000000,2019-06-07 00:00:00+00:00,118.25,9.096154,12.511628,26.9,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41643381,595.384615,7740,13,55.000000,2019-06-08 00:00:00+00:00,118.25,9.096154,12.511628,26.9,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41643530,1500.000000,6000,4,77.700000,2019-06-09 00:00:00+00:00,129.50,32.375000,30.240000,50.4,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...
41643856,2148.000000,10740,5,52.793296,2019-06-10 00:00:00+00:00,157.50,31.500000,19.474860,58.1,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...


Feast uses [Great Expectations](https://docs.greatexpectations.io/docs/) as a validation engine and [ExpectationSuite](https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/core/expectation_suite/index.html#great_expectations.core.expectation_suite.ExpectationSuite) as a dataset's profile. Hence, we need to develop a function that will generate ExpectationSuite. This function will receive instance of [PandasDataset](https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/dataset/index.html?highlight=pandasdataset#great_expectations.dataset.PandasDataset) (wrapper around pandas.DataFrame) so we can utilize both Pandas DataFrame API and some helper functions from PandasDataset during profiling.

In [27]:
DELTA = 0.1  # controlling allowed window in fraction of the value on scale [0, 1]

@ge_profiler
def stats_profiler(ds: PandasDataset) -> ExpectationSuite:
    # simple checks on data consistency
    ds.expect_column_values_to_be_between(
        "avg_speed",
        min_value=0,
        max_value=60,
        mostly=0.99  # allow some outliers
    )
    
    ds.expect_column_values_to_be_between(
        "total_miles_travelled",
        min_value=0,
        max_value=500,
        mostly=0.99  # allow some outliers
    )
    
    # expectation of means based on observed values
    observed_mean = ds.trip_count.mean()
    ds.expect_column_mean_to_be_between("trip_count",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))
    
    observed_mean = ds.earned_per_hour.mean()
    ds.expect_column_mean_to_be_between("earned_per_hour",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))
    
    
    # expectation of quantiles
    qs = [0.5, 0.75, 0.9, 0.95]
    observed_quantiles = ds.avg_fare.quantile(qs)
    
    ds.expect_column_quantile_values_to_be_between(
        "avg_fare",
        quantile_ranges={
            "quantiles": qs,
            "value_ranges": [[None, max_value] for max_value in observed_quantiles]
        })                                     
    
    return ds.get_expectation_suite()

Testing our profiler function:

In [28]:
ds.get_profile(profiler=stats_profiler)

<GEProfile with expectations: [
  {
    "meta": {},
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "avg_speed",
      "min_value": 0,
      "max_value": 60,
      "mostly": 0.99
    }
  },
  {
    "meta": {},
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "total_miles_travelled",
      "min_value": 0,
      "max_value": 500,
      "mostly": 0.99
    }
  },
  {
    "meta": {},
    "expectation_type": "expect_column_mean_to_be_between",
    "kwargs": {
      "column": "trip_count",
      "min_value": 10.387244591346153,
      "max_value": 12.695521167200855
    }
  },
  {
    "meta": {},
    "expectation_type": "expect_column_mean_to_be_between",
    "kwargs": {
      "column": "earned_per_hour",
      "min_value": 52.32062497564023,
      "max_value": 63.9474305257825
    }
  },
  {
    "meta": {},
    "expectation_type": "expect_column_quantile_values_to_be_between",
    "kwargs": {
      "col

**Verify that all expectations that we coded in our profiler are present here. Otherwise (if you can't find some expectations) it means that it failed to pass on the reference dataset (do it silently is default behavior of Great Expectations).**

Now we can create validation reference from dataset and profiler function:

In [29]:
validation_reference = ds.as_reference(profiler=stats_profiler)

and test it against our existing retrieval job

In [30]:
_ = job.to_df(validation_reference=validation_reference)

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/infra/offline_stores/offline_store.py:93: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


Validation successfully passed as no exception were raised.

### 5. Validating new historical retrieval 

Creating new timestamps for Dec 2020:

In [31]:
from feast.dqm.errors import ValidationFailed

In [32]:
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2020-12-01", "2020-12-07", freq='D')

In [33]:
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-05
...,...,...
35443,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-03
35444,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-04
35445,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-05
35446,7ebf27414a0c7b128e7925e1da56d51a8b81484f7630cf...,2020-12-06


In [34]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

Execute retrieval job with validation reference:

In [35]:
try:
    df = job.to_df(validation_reference=validation_reference)
except ValidationFailed as exc:
    print(exc.validation_report)

/Users/kike/opt/miniconda3/envs/feast/lib/python3.8/site-packages/feast/infra/offline_stores/offline_store.py:93: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


[
  {
    "success": false,
    "meta": {},
    "expectation_config": {
      "meta": {},
      "expectation_type": "expect_column_mean_to_be_between",
      "kwargs": {
        "column": "trip_count",
        "min_value": 10.387244591346153,
        "max_value": 12.695521167200855,
        "result_format": "COMPLETE"
      }
    },
    "result": {
      "observed_value": 6.692920555429092,
      "element_count": 4393,
      "missing_count": null,
      "missing_percent": null
    },
    "exception_info": {
      "raised_exception": false,
      "exception_message": null,
      "exception_traceback": null
    }
  },
  {
    "success": false,
    "meta": {},
    "expectation_config": {
      "meta": {},
      "expectation_type": "expect_column_mean_to_be_between",
      "kwargs": {
        "column": "earned_per_hour",
        "min_value": 52.32062497564023,
        "max_value": 63.9474305257825,
        "result_format": "COMPLETE"
      }
    },
    "result": {
      "observed_value": 6

Validation failed since several expectations didn't pass:
* Trip count (mean) decreased more than 10% (which is expected when comparing Dec 2020 vs June 2019)
* Average Fare increased - all quantiles are higher than expected
* Earn per hour (mean) increased more than 10% (most probably due to increased fare)